In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
dataset.info()

# Data preprocessing

In [ ]:
# Convert categorical columns to numerical
dataset = pd.get_dummies(dataset)
#dataset['expenses'] = np.log1p(dataset['expenses'])
dataset

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split the data into training and testing datasets
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

features = dataset.drop('expenses', axis=1)
labels = dataset['expenses']

# Split the data into training and testing datasets using Scikit-learn
train_dataset, test_dataset, train_labels, test_labels = train_test_split(features, labels, test_size=0.2, random_state=0)

# Scale the data
scaler = StandardScaler()
train_dataset = scaler.fit_transform(train_dataset)
test_dataset = scaler.transform(test_dataset)



# Train the Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU

# Define a simple neural network model for regression
model = Sequential([
    Dense(32, activation='relu', input_shape=[train_dataset.shape[1]]),
    Dense(16, activation='relu'),
    Dense(1)  # Single output layer for regression
])

# Compile the model with an appropriate loss function and optimizer
model.compile(optimizer='adam', loss='mae', metrics=['mae', 'mse'])

# Train the model
history = model.fit(train_dataset, train_labels, epochs=200, validation_split=0.2, verbose=2)


In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.

# Reverse the log transformation on the test labels
#test_labels = np.expm1(test_labels)
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()
#test_predictions = np.expm1(test_predictions) # Reverse the log transformation

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)


In [ ]:
test_predictions